In [3]:
number=5
print(number.__class__)
print(number.__class__.__class__)
print(number.__class__.__class__.__class__)

<class 'int'>
<class 'type'>
<class 'type'>


In [94]:
class IsIntegar_metaclass(type):
    def __new__(mcs, name, bases, attrs):
        print(mcs)
        print(name)
        print(bases)
        print(attrs)
        return super().__new__(mcs, name, bases, attrs)

    def __call__(cls, *args, **kwargs):
        if type(args[0]) != int:
            return False
        return type.__call__(cls, *args, **kwargs)

In [95]:
class IsIntegar(metaclass=IsIntegar_metaclass):
    def __init__(self, number):
        self.number = number

<class '__main__.IsIntegar_metaclass'>
IsIntegar
()
{'__module__': '__main__', '__qualname__': 'IsIntegar', '__init__': <function IsIntegar.__init__ at 0x0000023DD6925790>}


In [98]:
num1 = IsIntegar(5)
print(type(num1))
print(num1.number)

num2 = IsIntegar(5.1)
print(type(num2))
print(num2)

<class '__main__.IsIntegar'>
5
<class 'bool'>
False


In [100]:
import threading

In [103]:
# 加锁，实现线程安全
def synchronized(func):
    """
    Decorator in order to achieve thread-safe singleton class.
    """
    func.__lock__ = threading.Lock()

    def lock_func(*args, **kwargs):
        with func.__lock__:
            return func(*args, **kwargs)

    return lock_func


class SingleInstanceMetaClass(type):

    def __call__(cls, *args, **kwargs):
        if cls.instance:
            return cls.instance
        cls.instance = cls.__new__(cls)
        cls.instance.__init__(*args, **kwargs)
        return cls.instance

    @synchronized
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls, *args, **kwargs)


In [104]:
class A(metaclass=SingleInstanceMetaClass):
    pass

In [105]:
print(id(A()))
print(id(A()))

2464605115200
2464605115200


In [106]:
import time

In [146]:
class Singleton(type):
    def __call__(cls, *args, **kwargs):
        if hasattr(cls, "_instance"):
            return cls._instance
        cls._instance = super().__call__(*args, **kwargs)
        return cls._instance


class A(metaclass=Singleton):
    def __init__(self):
        time.sleep(1)

In [147]:
def task():
    a = A()
    print(id(a))

In [148]:
for i in range(10):
        t = threading.Thread(target=task)
        t.start()

0.002992868423461914
24646064498722464605271232
24646050195842464605272384
2464604962672
2464605506192
2464604994480
2464605507344
2464604986192


2464605503792


In [145]:
class Singleton(type):
    _lock = threading.Lock()
    def __call__(cls, *args, **kwargs):
        with cls._lock:
            if hasattr(cls, "_instance"):
                return cls._instance
            cls._instance = super().__call__(*args, **kwargs)
        return cls._instance


class A(metaclass=Singleton):
    def __init__(self):
        time.sleep(1)
        
def task():
    a = A()
    print(id(a))
    
for i in range(10):
        t = threading.Thread(target=task)
        t.start()

2464605035248
2464605035248
2464605035248
2464605035248
2464605035248
2464605035248
2464605035248
2464605035248
2464605035248
2464605035248
1.0291383266448975


In [ ]:
class Singleton(type):
    _lock = threading.Lock()
    
    def __call__(cls, *args, **kwargs):
        time.sleep(1)
        with cls._lock:
            if not hasattr(cls, "_instance"):
                cls._instance = super().__call__(*args, **kwargs)
        return cls._instance


class Person(metaclass=Singleton):
    def __init__(self,name, age):
        self.name = name
        self.age = age


def task(arg):
    p = Person('张三',18)
    print(p)


if __name__ == '__main__':
    for i in range(10):
        t = threading.Thread(target=task, args=[i,])
        t.start()

In [3]:
a=3
if a==1 or 2:
    print(123)

123
